In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2023-11-11 23:53:01.442609: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-11 23:53:01.685707: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 23:53:01.685743: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 23:53:01.686677: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-11 23:53:01.790222: I tensorflow/core/platform/cpu_feature_g

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Número de GPUs disponíveis:", len(physical_devices))

Número de GPUs disponíveis: 1


2023-11-11 23:53:05.043440: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-11 23:53:05.066459: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-11 23:53:05.066508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [4]:
from tensorflow.keras.preprocessing import sequence

In [6]:
import pandas as pd
import numpy as np

In [7]:
seed = 777
np.random.seed(seed)

In [8]:
df_1 = pd.read_csv('datasets/BERT_Embedding_1.csv')
df_2 = pd.read_csv('datasets/BERT_Embedding_2.csv')

df = pd.concat([df_1,df_2])
df.reset_index(inplace=True)

In [118]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset


embeddings = df.iloc[:,8:].values
labels = df['Class']

# Supondo que você tenha uma coluna 'rotulo' em seu DataFrame
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=seed)

# Convertendo para tensores do PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Criando datasets e dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [123]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Ajuste nesta linha
        return out

# Defina os parâmetros do modelo
input_size = 768  # número de embeddings
hidden_size = 64  # tamanho do estado oculto
output_size = 3  # substitua num_classes pelo número real de classes

# Crie uma instância do modelo
model = LSTMModel(input_size, hidden_size, output_size)


In [124]:
import torch.optim as optim

# Defina a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento
num_epocas = 100

for epoca in range(num_epocas):
    model.train()
    optimizer.zero_grad()

    saida = model(X_train_tensor)
    perda = criterion(saida, y_train_tensor)
    
    perda.backward()
    optimizer.step()

    print(f'Época [{epoca + 1}/{num_epocas}], Perda: {perda.item():.4f}')



Época [1/100], Perda: 1.1019
Época [2/100], Perda: 1.1014
Época [3/100], Perda: 1.0970
Época [4/100], Perda: 1.0929
Época [5/100], Perda: 1.0911
Época [6/100], Perda: 1.0897
Época [7/100], Perda: 1.0871
Época [8/100], Perda: 1.0838
Época [9/100], Perda: 1.0807
Época [10/100], Perda: 1.0783
Época [11/100], Perda: 1.0755
Época [12/100], Perda: 1.0718
Época [13/100], Perda: 1.0682
Época [14/100], Perda: 1.0654
Época [15/100], Perda: 1.0624
Época [16/100], Perda: 1.0592
Época [17/100], Perda: 1.0568
Época [18/100], Perda: 1.0545
Época [19/100], Perda: 1.0519
Época [20/100], Perda: 1.0500
Época [21/100], Perda: 1.0483
Época [22/100], Perda: 1.0461
Época [23/100], Perda: 1.0444
Época [24/100], Perda: 1.0424
Época [25/100], Perda: 1.0406
Época [26/100], Perda: 1.0393
Época [27/100], Perda: 1.0373
Época [28/100], Perda: 1.0354
Época [29/100], Perda: 1.0339
Época [30/100], Perda: 1.0321
Época [31/100], Perda: 1.0307
Época [32/100], Perda: 1.0291
Época [33/100], Perda: 1.0273
Época [34/100], Per

In [125]:
# Avaliação no conjunto de teste
model.eval()

with torch.no_grad():
    saida_treino = model(X_train_tensor)
    _, predicoes_treino = torch.max(saida_treino, 1)
    acuracia_treino = torch.sum(predicoes_treino == y_train_tensor).item() / len(y_train_tensor)

    saida_teste = model(X_test_tensor)
    _, predicoes_teste = torch.max(saida_teste, 1)
    acuracia_teste = torch.sum(predicoes_teste == y_test_tensor).item() / len(y_test_tensor)

In [126]:
print(acuracia_treino)
print(acuracia_teste)

0.5310416666666666
0.5175
